<a href="https://colab.research.google.com/github/fakabbir/AI-Playbook/blob/master/Glove%20%2B%20GCN%20%2B%20CoreNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Downloading the vector file
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-04-20 08:34:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-20 08:34:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-20 08:34:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
# Checking cuda version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
# Downloading the right cude version pytorch
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu111

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu111


In [6]:
# Checking files
!ls -a

.   .config	       glove.6B.200d.txt  glove.6B.50d.txt  sample_data
..  glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip


In [7]:
# Converting into vector space
import os
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [8]:
embeddings_index['dancing']

array([ 8.8927e-01,  1.3668e-01,  1.0018e-01,  8.6889e-02, -3.7379e-04,
        1.2549e+00, -3.0605e-01,  8.2326e-01, -3.1645e-01, -2.6105e-01,
        8.8670e-02, -3.7473e-01,  2.3649e-01,  1.4771e-01, -3.5600e-01,
        1.3845e-01, -1.6199e-01,  4.6586e-01,  2.3392e-01, -3.7496e-01,
       -1.4306e-01,  1.6502e-01,  7.2604e-01, -5.7993e-01,  2.1166e-01,
        6.6273e-01, -7.3482e-01, -2.5190e-01,  4.5720e-01, -4.2118e-02,
       -8.2452e-01, -2.2596e-01,  4.6759e-01, -1.9811e-01,  5.5191e-01,
       -4.3812e-01, -4.5375e-02,  2.4324e-01, -2.0800e-01, -6.9571e-01,
        1.8216e-01,  3.7280e-01, -6.7986e-01,  1.0618e-01,  2.1276e-02,
       -2.1635e-01, -1.1486e+00, -8.0755e-03,  1.2753e-01, -6.2336e-01,
       -6.1599e-01, -3.0707e-01,  9.8371e-03,  9.4887e-01, -2.1619e-01,
       -2.0712e+00, -1.1367e-01,  7.9654e-01,  2.3070e-01,  7.2508e-01,
        3.6047e-01,  1.2606e+00,  6.0255e-03,  1.5010e-01, -2.1836e-01,
        7.9186e-01,  6.6293e-01, -7.5083e-01,  1.0351e-01, -2.03

In [9]:
from torch import nn
import torch

In [10]:
# Prepare embedding matrix
num_tokens = 1 + len(embeddings_index)
embedding_dim = len(embeddings_index['the'])
embedding_matrix = np.zeros((num_tokens, embedding_dim))

embedding_matrix_idx = {}
for idx,word in enumerate(embeddings_index):
    embedding_matrix[1+idx] = embeddings_index[word]
    embedding_matrix_idx[word] = 1 +idx

weight = torch.FloatTensor(embedding_matrix)
embedding = nn.Embedding.from_pretrained(weight)

In [11]:
params = {
            'hidden_size': 192,
            'input_size': embedding_dim,
            'bias': True,
            'dropout': 0.2,
            'bidirectional': True }

model = nn.LSTM(**params)
fc = nn.Linear(192, 2)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [12]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, 150)
        self.fc2 = nn.Linear(150, output_dim)
        
    def forward(self, text):
        #text = [sent len, batch size]
        idx = embedding_matrix_idx[text]
        embedded = torch.unsqueeze(torch.unsqueeze(weight[idx], dim=0), dim = 0)
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        # assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        return self.fc2(self.fc1(hidden.squeeze(0)))

rnn = RNN(1,100,192,2)
rnn("cat")

tensor([[-0.0707, -0.1051]], grad_fn=<AddmmBackward0>)

In [13]:
# Given a sentance convert it into its idx form

x = "This is a cool thing"

def pipeline(x):
    x = x.lower()
    x = x.split(" ")
    vector = []
    for i in x:
      if i in embedding_matrix_idx:
        vector.append(embedding_matrix_idx[i])
      else:
        vector.append(0)

    return vector
pipeline(x)


[38, 15, 8, 3452, 874]

In [37]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        torch.manual_seed(1234567)
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, 150)
        self.fc2 = nn.Linear(150, output_dim)
        # self.sig = nn.Sigmoid()
        
    def forward(self, text):
        #text = [sent len, batch size]
        text = pipeline(text)
        n = len(text)
        text = torch.tensor(text)
        embedded =  torch.unsqueeze(self.embedding(text), dim = 0)
        #embedded = [sent len, batch size, emb dim]

        
        hidden = torch.zeros(1,1,100)
        for i in embedded:
          output, hidden = self.rnn(hidden)

        # opt,hdn = self.rnn(embedded.view(len(n), 1, -1))
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        # assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        out =  self.fc2(self.fc1(hidden.squeeze(0)))
        # sigmoid function
        # sig_out = self.sig(out)

        return torch.argmax(out)

rnn = RNN(1,100,192,2)
rnn("run run run please follow")

tensor(0)

In [38]:
import torch.optim as optim
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn.parameters(), lr=0.1)

In [39]:
training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple", 1),
    ("Everybody read that book", 0)
]

In [40]:
with torch.no_grad():
    inputs = training_data[0][0]
    print(inputs)
    tag_scores = rnn(inputs)
    print(tag_scores)

The dog ate the apple
tensor(0)
